In [5]:
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

Se realizo una extraccion total de los datos del reto ["American Express -Default Predition"](https://www.kaggle.com/competitions/amex-default-prediction/overview), el cual tiene el fin de determinar si un cliente sera propenso a pagar su estado de cuenta o por el contrario caeera en un inclumplimiento de su pago. 

Para realizar el entrenamiento hace falta los datos de train_data.csv que contienen el comportamiento de los clientes segmentados en categorias, los cuales relizaran el rol de variables depedientes. A su vez sera necesario los datos en train_labels.csv que contienen los resultados del cumplimiento de los pagos de los clientes. 

Los datos no se encuentran disponibles en el repositorio por cuestion de su dimencion, por lo que para ejecutar el codigo es necesario bajar manualmente ["train_data.csv"](https://www.kaggle.com/competitions/amex-default-prediction/data?select=train_data.csv) y ["train_labels.csv"](https://www.kaggle.com/competitions/amex-default-prediction/data?select=train_labels.csv) en el directorio "./data".

In [6]:
trainData = dd.read_csv('./data/train_data.csv', blocksize=50e6)
trainLabels = dd.read_csv('./data/train_labels.csv', blocksize=50e6)

Se uso dask para seccionar los datos de train_data.csv en bloques de 50 MB para poder ser posible su manejo.

In [7]:
trainData

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
npartitions=327,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [8]:
trainLabels

,customer_ID,target
npartitions=1,,
,object,int64
,...,...


# Tranformacion de los datos

In [9]:
trainSet = trainData.merge(trainLabels, how="left", on=["customer_ID"])
trainSet

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
npartitions=327,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [10]:
trainSet.describe().compute()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
count,5.485466e+06,5.531451e+06,5.531451e+06,5.529435e+06,5.531451e+06,4.510907e+06,5.529435e+06,5.529435e+06,791314.000000,3.873055e+06,...,1.946990e+05,1.946990e+05,5.429903e+06,5.490819e+06,5.429903e+06,944408.000000,5.429903e+06,5.490724e+06,5.429903e+06,5.531451e+06
mean,6.563340e-01,1.531172e-01,1.240100e-01,6.214887e-01,7.880270e-02,2.258455e-01,5.978469e-02,1.325389e-01,0.184974,1.546841e-01,...,1.424409e-02,1.646183e-01,1.789305e-01,2.664348e-02,1.645212e-01,0.390799,1.788022e-01,5.238952e-02,6.233496e-02,2.490972e-01
std,2.446494e-01,2.700709e-01,2.119869e-01,4.014877e-01,2.263971e-01,1.933475e-01,2.025443e-01,2.349929e-01,0.228185,2.133977e-01,...,9.571115e-02,2.670208e-01,3.790614e-01,1.455480e-01,3.482771e-01,0.236182,3.789498e-01,1.825135e-01,1.934937e-01,4.324903e-01
min,-4.589548e-01,5.026190e-09,-7.588799e+00,9.192280e-09,1.534223e-09,-6.271320e-01,5.566545e-10,6.285293e-09,-0.000454,1.154550e-07,...,1.078787e-08,3.307923e-08,3.767347e-10,3.725073e-09,1.650100e-10,-0.014539,5.549692e-09,2.500991e-09,1.226024e-09,0.000000e+00
25%,5.064879e-01,4.771657e-03,9.758679e-03,1.478242e-01,3.016002e-03,1.326595e-01,2.999373e-03,5.556932e-03,0.056150,4.590660e-02,...,3.099202e-03,4.944534e-03,3.178373e-03,2.648601e-03,3.191714e-03,0.259432,3.201604e-03,2.911429e-03,3.178664e-03,0.000000e+00
50%,7.189715e-01,9.429229e-03,3.599524e-02,8.154153e-01,5.958978e-03,1.678345e-01,5.926710e-03,1.117388e-02,0.151166,9.578324e-02,...,5.702148e-03,8.609538e-03,6.303189e-03,5.238345e-03,6.300143e-03,0.447707,6.270650e-03,5.715328e-03,6.319477e-03,0.000000e+00
75%,8.796860e-01,2.671200e-01,1.552360e-01,1.003166e+00,8.849452e-03,2.811598e-01,8.844268e-03,1.921618e-01,0.302292,2.045041e-01,...,8.042659e-03,5.053661e-01,9.417924e-03,7.792673e-03,9.453466e-03,0.606487,9.429204e-03,8.520661e-03,9.444622e-03,1.000000e+00
max,1.010000e+00,5.389619e+00,1.324060e+00,1.010000e+00,3.256284e+00,5.482888e+00,8.988807e+00,1.625262e+00,4.191119,1.011162e+01,...,1.009998e+00,3.005383e+00,1.010000e+00,1.010000e+00,1.339910e+00,2.229368,1.010000e+00,1.343331e+00,4.827630e+00,1.000000e+00


Se unieron los dos sets de datos en un unico set para el entrenamiento.

# Limpieza de datos

In [11]:
trainSet.isna().compute()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16917,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
16918,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
16919,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
16920,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False


In [12]:
len(trainSet)

5531451

El metodo isna() nos servira para detectar las columnas que poseen valores NaN, mientras que el metodo len nos servira para determinar si lo mas optimo sera eliminar las filas, imputar los datos faltantes o eliminar l columna.

In [13]:
nansColumns = trainSet.columns[trainSet.isna().compute().any()].tolist()

En nansColumns obtenemos la lista de columnas que poseen valores NaN.

In [14]:
len(nansColumns)/len(trainSet.columns)

0.6387434554973822

Determinamos que el 64% de las columnas poseen valores NaN.

In [15]:
dropColumns = trainSet.columns[trainSet.isna().compute().sum()/len(trainSet) >= .2].tolist() 
dropColumns

['D_42',
 'D_43',
 'D_46',
 'D_49',
 'D_50',
 'D_53',
 'D_56',
 'S_9',
 'B_17',
 'D_66',
 'D_73',
 'D_76',
 'D_77',
 'R_9',
 'D_82',
 'B_29',
 'D_87',
 'D_88',
 'D_105',
 'D_106',
 'R_26',
 'D_108',
 'D_110',
 'D_111',
 'B_39',
 'S_27',
 'B_42',
 'D_132',
 'D_134',
 'D_135',
 'D_136',
 'D_137',
 'D_138',
 'D_142']

En dropColumns obtenemos la lista de columnas que vamos a eliminar por tener 20% o mas de datos NaN. 

In [16]:
trainSet = trainSet.drop(columns=dropColumns)

Eliminamos las columnas presentes en dropColumns del dataset.

In [17]:
nansColumns = trainSet.columns[trainSet.isna().compute().any()].tolist()

In [18]:
len(nansColumns)/len(trainSet.columns) #0.6387434554973822, 0.5605095541401274

0.5605095541401274

Al volver a calcular el porcentaje de columnas que poseen valores NaN, podemos obsevar que hemos reducido la cantidad de columnas con valores NaN en un 8%.

In [19]:
trainSet.to_csv('./data/temp/20_n_n/')

['/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/000.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/001.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/002.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/003.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/004.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/005.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/006.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/007.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/008.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/009.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/010.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/011.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/012.part',
 '/home/alt9193/Documents/IA/IA_Avanzada/data/temp/20_n_n/013.part',
 '/home/alt9193/Documents/IA/IA_Av

Guardamos el progreso de la limpieza de los datos.

In [20]:
trainSet = dd.read_csv('./data/temp/20_n_n/*')

Nos aseguramos de estar usando los datos de la limpieza guardada.

In [21]:
dropRows = trainSet.columns[trainSet.isna().compute().sum()/len(trainSet) <= .05].tolist() 
dropRows

['Unnamed: 0',
 'customer_ID',
 'S_2',
 'P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'D_41',
 'B_3',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_47',
 'B_6',
 'B_7',
 'B_8',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'B_10',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'B_12',
 'S_8',
 'D_55',
 'B_13',
 'R_5',
 'D_58',
 'B_14',
 'D_59',
 'D_60',
 'B_15',
 'S_11',
 'D_63',
 'D_64',
 'D_65',
 'B_16',
 'B_18',
 'B_19',
 'B_20',
 'D_68',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'P_4',
 'D_74',
 'D_75',
 'B_24',
 'R_7',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'B_30',
 'S_18',
 'D_86',
 'R_17',
 'R_18',
 'B_31',
 'S_19',
 'R_19',
 'B_32',
 'S_20',
 'R_20',
 'R_21',
 'B_33',
 'D_89',
 'R_22',
 'R_23',
 'D_91',
 'D_92',
 'D_93',
 'D_94',
 'R_24',
 'R_25',
 'D_96',
 'S_22',
 'S_23',
 'S_24',
 'S_25',
 'S_26',
 '

En dropRows obtenemos las columnas que poseen 5% o menos de valores NaN, las cuales serviran como subset para eliminar las filas correspondientes. 

In [25]:
trainSet.dropna(subset=dropRows)

,Unnamed: 0,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_44,B_4,D_45,B_5,R_2,D_47,D_48,B_6,B_7,B_8,D_51,B_9,R_3,D_52,P_3,B_10,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,B_13,R_5,D_58,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_18,B_19,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,P_4,D_74,D_75,B_24,R_7,B_25,B_26,D_78,D_79,R_8,S_16,D_80,R_10,R_11,B_27,D_81,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_30,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_107,B_36,B_37,R_27,B_38,D_109,D_112,B_40,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
npartitions=327,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [26]:
nansColumns = trainSet.columns[trainSet.isna().compute().any()].tolist()

In [27]:
len(nansColumns)/len(trainSet.columns) #0.6387434554973822, 0.5605095541401274, 0.5569620253164557

0.5569620253164557

In [29]:
trainSet.to_csv('./data/temp/20_n_5/')

KeyboardInterrupt: 

Guardamos el progreso de la limpieza de los datos.

In [ ]:
trainSet = dd.read_csv('./data/temp/20_n_5/*')

Nos aseguramos de estar usando los datos de la limpieza guardada.

In [22]:
#from dataprep.eda import create_report
#report = create_report(trainSet, title='AMEX Train report')

In [23]:
#import seaborn as sns
#sns.set_theme()

In [24]:
#ax = sns.heatmap(trainSet)